In [1]:
from StopGoClassifier import StopGoClassifier
from datetime import datetime
from sklearn import metrics
import seaborn as sns
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [2]:
dataset_path = './groundTruthGenerator/groundTruth'

### Test / Validation

In [3]:
move_stop_test = pd.read_csv(f'{dataset_path}/stop_test.csv')
move_stop_test['index'] = move_stop_test.index
veh_id_unique = move_stop_test['id'].unique()

In [6]:
ac_list = []
pr_list = []
re_list = []
f1_list = []
model = StopGoClassifier()
data = move_stop_test
for veh_id in veh_id_unique:
    staga = data[data['id'] == veh_id]
    y_true = staga['stop']
    X = staga.drop(columns=['stop'])
    X['ts'] = X['timestep'].apply(lambda t: datetime.fromtimestamp(t))
    X['ts'] = pd.to_datetime(X['timestep'], utc=True)
    X = X.set_index('ts').tz_convert(None).reset_index()
    model.read(X['ts'], X['x'], X['y'])
    stop_intervals = model.run()

    pred = X[['timestep']].copy()
    pred['stop'] = False
    for i in stop_intervals.values:
        start, stop = i[0], i[1]
        interval = pred['timestep'].between(start, stop, inclusive='both')
        pred[interval] = True

    ac_list.append(metrics.accuracy_score(y_true, pred['stop']))
    pr_list.append(metrics.precision_score(y_true, pred['stop']))
    re_list.append(metrics.recall_score(y_true, pred['stop']))
    f1_list.append(metrics.f1_score(y_true, pred['stop']))

ac_mean = np.mean(ac_list)
pr_mean = np.mean(pr_list)
re_mean = np.mean(re_list)
f1_mean = np.mean(f1_list)

print('Accuracy mean:', ac_mean)
print('Precision mean:', pr_mean)
print('Recall mean:', re_mean)
print('F1 mean:', f1_mean)

Accuracy mean: 0.9624194799923896
Precision mean: 0.0
Recall mean: 0.0
F1 mean: 0.0
